# Configure Preprocessing Params

This notebook should be used to set up preprocessing params.
Cells marked with `SET PARAMETERS` contain crucial variables that need to be set according to your specific experimental setup and data organization.
Please review and modify these variables as needed before proceeding with the analysis.

## Imports

In [1]:
import re
from pathlib import Path
from itertools import chain

import pandas as pd
import yaml
from microfilm.microplot import Microimage
import matplotlib.pyplot as plt

from lib.shared.configuration_utils import CONFIG_FILE_HEADER, create_samples_df, create_micropanel
from lib.preprocess.preprocess import extract_tile_metadata, nd2_to_tiff
from lib.preprocess.file_utils import get_sample_fps

## SET PARAMETERS

### Fixed parameters for preprocessing

- `CONFIG_FILE_PATH`: Path to a Brieflow config file used during processing*.
- `ROOT_FP`: Path to root of Brieflow output directory*.

*Note: Paths can be absolute or relative to where workflows are run from.

In [2]:
CONFIG_FILE_PATH = "config/config.yml"
ROOT_FP = "analysis_root/"

## SET PARAMETERS

### Paths to dataframes with sample information

- `SBS_SAMPLES_DF_FP`/`PHENOTYPE_SAMPLES_DF_FP`: Path to dataframe where SBS/phenotype samples location and metadata will be stored*.
- `SBS_DATA_FP`/`PHENOTYPE_DATA_FP`: Path to directories with SBS/phenotype sample nd2 files*.
- `SBS_SAMPLE_PATTERN`/`PHENOTYPE_SAMPLE_PATTERN`: Regex patterns to match SBS/phenotype sample nd2 files.
- `SBS_CHANNEL_PATTERN`/`PHENOTYPE_CHANNEL_PATTERN`: Optional regex patterns to extract channel information from filenames. Set to None if files are multichannel or if channel information is not needed. If two rounds of multichannel images are generated, use distinguishing features between these to get channel variables.

*Note: Paths can be absolute or relative to where workflows are run from.

In [ ]:
# paths to sample dataframes
SBS_SAMPLES_DF_FP = "config/sbs_samples.tsv"
PHENOTYPE_SAMPLES_DF_FP = "config/phenotype_samples.tsv"

# directories with SBS/phenotype sample nd2 files
SBS_REAL_IMAGES_FP = Path("data/real_images/sbs/")
SBS_EMPTY_IMAGES_FP = Path("data/empty_images/sbs/")
PHENOTYPE_REAL_IMAGES_FP = Path("data/real_images/phenotype/")
PHENOTYPE_EMPTY_IMAGES_FP = Path("data/empty_images/phenotype/")

# Patterns to match SBS/phenotype sample nd2 files
SBS_SAMPLE_PATTERN = r"C(\d+)_Wells-([A-Z]\d+)_Points-(\d+)"
SBS_METADATA = ["cycle", "well", "tile"]
PHENOTYPE_SAMPLE_PATTERN = r"Wells-([A-Z]\d+)_Points-(\d+)"
PHENOTYPE_METADATA = ["well", "tile"]

You must supply a working regex to the _PATTERN variables. If you don't have experience with regex, you can use the following LLM prompt to generate the example sample patterns.

_Enter into a basic LLM chatbot_: Given ND2 filenames from your experiment, generate regex patterns to extract metadata. Return only the regex patterns with no explanation.

Example filenames:
[ENTER YOUR EXAMPLE FILES HERE]

Required regex patterns (return these exact variable assignments):

SBS_SAMPLE_PATTERN = r"..."
PHENOTYPE_SAMPLE_PATTERN = r"..."
SBS_CHANNEL_PATTERN = r"..." 
PHENOTYPE_CHANNEL_PATTERN = r"..."

The patterns should extract:

1. SBS_SAMPLE_PATTERN:
  - Cycle number (after "C")
  - Well ID (after "Wells-")  
  - Tile number (after "Points-")

2. PHENOTYPE_SAMPLE_PATTERN:
  - Well ID (after "Wells-")
  - Tile number (after "Points-") 

3. SBS_CHANNEL_PATTERN:
  - Channel name (between "Channel_" and next "_" or ".")
  Set to None if images are multichannel

4. PHENOTYPE_CHANNEL_PATTERN:  
  - Channel name (between "Channel_" and next "_" or ".")
  Set to None if images are multichannel

The patterns will be used like this:
```python
# Sample pattern matching
match = re.search(SBS_SAMPLE_PATTERN, filename)
cycle = int(match.group(1))
well = match.group(2) 
tile = int(match.group(3))

# Channel pattern matching (if not None)
if SBS_CHANNEL_PATTERN:
   channel_match = re.search(SBS_CHANNEL_PATTERN, filename)
   channel = channel_match.group(1)
```

## Create Sample DFs

## Test Metadata Extraction

In [5]:
sbs_samples_real = create_samples_df(SBS_REAL_IMAGES_FP, SBS_SAMPLE_PATTERN, SBS_METADATA)
display(sbs_samples_real)

create_samples_df(PHENOTYPE_REAL_IMAGES_FP, PHENOTYPE_SAMPLE_PATTERN, PHENOTYPE_METADATA)

No image directory provided, returning empty samples DataFrame!


,sample_fp


,sample_fp,well,tile
0,data/real_images/phenotype/P001_Pheno_20x_Well...,A1,1
1,data/real_images/phenotype/P001_Pheno_20x_Well...,A1,100
2,data/real_images/phenotype/P001_Pheno_20x_Well...,A2,1
3,data/real_images/phenotype/P001_Pheno_20x_Well...,A2,100


## SET PARAMETERS

- `SBS_CHANNEL_ORDER`/`PHENOTYPE_CHANNEL_ORDER`: Manually set channel order _if ND2 images are acquired as single channels, or there are multiple files for each tile (e.g. multiple rounds of phenotype images). Should be `None` if multichannel image files are acquired. The extracted channel names must match the values that will be displayed in the samples DataFrame channel column (e.g., `["DAPI", "GFP", "CY3", "CY5", "AF750"]`).

**Note** For single-channel files, each file must contain a channel identifier that your regex can extract. For multichannel files, set the channel patterns to `None`. Metadata extraction is only performed on the first channel dimension for each tile. Please ensure the Dapi channel is displayed first!

In [ ]:
SBS_CHANNEL_ORDER = None
PHENOTYPE_CHANNEL_ORDER = None

In [ ]:
# Extract metadata from test sbs sample
test_sbs_metadata = extract_tile_metadata(
    sbs_samples["sample_fp"][0], 
    sbs_samples["tile"][0], 
    verbose=True
)
print("SBS test metadata:")
display(test_sbs_metadata)

# Extract metadata from test phenotype sample
test_phenotype_metadata = extract_tile_metadata(
    phenotype_samples["sample_fp"][0],
    phenotype_samples["tile"][0],
    verbose=True,
)
print("Phenotype test metadata:")
display(test_phenotype_metadata)

## SET PARAMETERS

### ND2 conversion

- `SBS_CHANNEL_ORDER_FLIP`/`PHENOTYPE_CHANNEL_ORDER_FLIP`: Whether or not to flip channel order when converting ND2->tiff. Should be `False` if channels are in a standard order (with Dapi first), or `True` if channels are reversed. This will only occur for multichannel ND2 files, for each individual ND2 file. Setting the channel order for single channel files is done by setting `SBS_CHANNEL_ORDER`/`PHENOTYPE_CHANNEL_ORDER` previously.

**Note** Channel order can be checked with the test conversions below. Please ensure the Dapi channel is displayed first!

In [ ]:
SBS_CHANNEL_ORDER_FLIP = True
PHENOTYPE_CHANNEL_ORDER_FLIP = True

## Test ND2 Conversion

### Test SBS conversion

In [ ]:
sbs_sample = get_sample_fps(sbs_samples, 
                            well=sbs_samples["well"][0], 
                            tile=sbs_samples["tile"][0], 
                            cycle=sbs_samples["tile"][0], 
                            channel_order = SBS_CHANNEL_ORDER)

phenotype_sample = get_sample_fps(phenotype_samples, 
                                  well=phenotype_samples["well"][0], 
                                  tile=phenotype_samples["tile"][0], 
                                  channel_order = PHENOTYPE_CHANNEL_ORDER)

In [ ]:
# Convert test sbs sample to tiff
sbs_image = nd2_to_tiff(
    sbs_sample, SBS_CHANNEL_ORDER, verbose=True
)
# Create micropanel to display converted samples
print("Converted SBS test sample:")
sbs_microimages = [Microimage(image) for image in sbs_image]
sbs_panel = create_micropanel(sbs_microimages, add_channel_label=False)
plt.show()

### Test phenotype conversion

In [ ]:
# Convert test phenotype sample to tiff
phenotype_image = nd2_to_tiff(
    phenotype_sample, PHENOTYPE_CHANNEL_ORDER_FLIP, verbose=True
)

# Create micropanel to display converted samples
print("Converted phenotype test sample:")
phenotype_microimages = [Microimage(image) for image in phenotype_image]
phenotype_panel = create_micropanel(phenotype_microimages, add_channel_label=False)
plt.show()

## Create config file with params

In [ ]:
# Create empty config variable
config = {}

# Add all section
config["all"] = {
    "root_fp": ROOT_FP,
}

# Add preprocess section
config["preprocess"] = {
    "sbs_samples_fp": SBS_SAMPLES_DF_FP,
    "phenotype_samples_fp": PHENOTYPE_SAMPLES_DF_FP,
    "sbs_channel_order": SBS_CHANNEL_ORDER,
    "phenotype_channel_order": PHENOTYPE_CHANNEL_ORDER,
    "sbs_channel_order_flip": SBS_CHANNEL_ORDER_FLIP,
    "phenotype_channel_order_flip": PHENOTYPE_CHANNEL_ORDER_FLIP,
}

# Write the updated configuration back with markdown-style comments
with open(CONFIG_FILE_PATH, "w") as config_file:
    # Write the introductory markdown-style comments
    config_file.write(CONFIG_FILE_HEADER)

    # Dump the updated YAML structure, keeping markdown comments for sections
    yaml.dump(config, config_file, default_flow_style=False)